In [1]:
#prepare 'tuples_train_shuffled.txt'
import re
train_tuple = open("tuples_train_posi.txt")
neg_train_tuple = open("tuples_train_neg.txt")

In [2]:
train_tuple_content = train_tuple.readlines()
neg_train_tuple_content = neg_train_tuple.readlines()

for i in range(len(train_tuple_content)):
    train_tuple_content[i] = train_tuple_content[i].strip('\r\n') + ' 1\r\n'
    train_tuple_content.append(neg_train_tuple_content[i].strip('\r\n') + ' 0\r\n')

In [3]:
import random
random.shuffle(train_tuple_content)

In [4]:
train_tuple_shuffled = open('tuples_train_shuffled.txt','w')

In [5]:
for i in range(len(train_tuple_content)):
    train_tuple_shuffled.write(train_tuple_content[i])

train_tuple.close()
neg_train_tuple.close()
train_tuple_shuffled.close()

In [6]:
#prepare 'tuples_test_shuffled.txt'
test_tuple = open("tuples_test_posi.txt")
neg_test_tuple = open("tuples_test_neg.txt")

In [7]:
test_tuple_content = test_tuple.readlines()
neg_test_tuple_content = neg_test_tuple.readlines()

for i in range(len(test_tuple_content)):
    test_tuple_content[i] = test_tuple_content[i].strip('\r\n') + ' 1\r\n'
    test_tuple_content.append(neg_test_tuple_content[i].strip('\r\n') + ' 0\r\n')

In [8]:
random.shuffle(test_tuple_content)

In [9]:
test_tuple_shuffled = open('tuples_test_shuffled.txt','w')

In [10]:
for i in range(len(test_tuple_content)):
    test_tuple_shuffled.write(test_tuple_content[i])
    
test_tuple.close()
neg_test_tuple.close()
test_tuple_shuffled.close()

In [2]:
#prepare lmdb & hdf5 for training data
import re
open('top_ind_test.txt')
tuples_train = open('tuples_train_shuffled.txt')
top_train = open("top_ind_train.txt").readlines() 
bot_train = open("bottom_ind_train.txt").readlines()
sho_train = open("shoe_ind_train.txt").readlines()
top_path = open("../img_list/img_list_top.txt").readlines()
bot_path = open("../img_list/img_list_bottom.txt").readlines()
sho_path = open("../img_list/img_list_shoe.txt").readlines()

In [5]:
#train tuples num
for i, l in enumerate(tuples_train):
            pass
print(i+1)

54000


In [6]:
import Image
import numpy as np

size = 300, 300

k = 0
count = 0

#initialization
#N = 54000
#data = np.zeros(N,9,300,300)
#labels = np.zeros(N,1)

for line in tuples_train:
    count = count + 1
    if count % 100 == 0:
        print("%d into LMDB......"%(count))
    #label 1-posi, 0-neg
    label = np.array([int(re.findall(r'\d+',line)[4])])
    #inter idx
    top_idx1 = re.findall(r'\d+',line)[1]
    bot_idx1 = re.findall(r'\d+',line)[2]
    sho_idx1 = re.findall(r'\d+',line)[3]
    #final idx
    top_idx2 = top_train[int(top_idx1)]
    bot_idx2 = bot_train[int(bot_idx1)]
    sho_idx2 = sho_train[int(sho_idx1)]
    #image path
    top_img = '../../Images/top/' + top_path[int(top_idx2)].strip('\r\n')
    bot_img = '../../Images/bottom/' + bot_path[int(bot_idx2)].strip('\r\n')
    sho_img = '../../Images/shoe/' + sho_path[int(sho_idx2)].strip('\r\n')
    #read images
    im_top = Image.open(top_img)
    im_bot = Image.open(bot_img)
    im_sho = Image.open(sho_img)
    #resize to the same size
    im_top = im_top.resize(size, Image.ANTIALIAS)
    im_bot = im_bot.resize(size, Image.ANTIALIAS)
    im_sho = im_sho.resize(size, Image.ANTIALIAS)
    #image to nparray
    im_top = np.array(im_top)
    im_bot = np.array(im_bot)
    im_sho = np.array(im_sho)
    #change type to np.int64, np.uint8 (lmdb)
    label.astype(np.int64)
    im_top.astype(np.uint8)
    im_bot.astype(np.uint8)
    im_sho.astype(np.uint8)
    #change type to np.float32 (hdf5)
    #label.astype(np.float32)
    #im_top.astype(np.float32)
    #im_bot.astype(np.float32)
    #im_sho.astype(np.float32)
    #from (height,width,channel) to (channel,height,width)
    im_top = np.rollaxis(im_top,2,0)
    im_bot = np.rollaxis(im_bot,2,0)
    im_sho = np.rollaxis(im_sho,2,0)
    #stack over "channel"
    im_outfit = np.concatenate((im_top, im_bot, im_sho),axis = 0)
    #from (channel,height,width) to (1,channel,height,width)
    label = label[np.newaxis, :]
    im_outfit = im_outfit[np.newaxis, :]
    #for stacking over num
    if k == 0:
        k = 1
        data = im_outfit
        labels = label
    else:
        data = np.concatenate((data, im_outfit),axis = 0)
        labels = np.concatenate((labels, label),axis = 0)

In [7]:
print(data.shape)
print(labels.shape)

NameError: name 'data' is not defined

In [14]:
#write to lmdb
import lmdb
caffe_root = '/home/tonghe2/caffe-master/'  # this file is expected to be in {caffe_root}/examples/siamese
import sys
sys.path.insert(0, caffe_root + 'python')
import caffe

In [15]:
map_size = data.nbytes * 10

In [19]:
env = lmdb.open('/home/tonghe2/train_lmdb', map_size=map_size)

In [20]:
with env.begin(write=True) as txn:     
    # txn is a Transaction object
    for i in range(labels.shape[0]):
        datum = caffe.proto.caffe_pb2.Datum()
        datum.channels = data.shape[1]
        datum.height = data.shape[2]
        datum.width = data.shape[3]
        datum.data = data[i].tobytes()
            
        datum.label = int(labels[i])
        str_id = '{:08}'.format(i)

In [74]:
#write to HDF5 (absolute path)
import h5py
with h5py.File('/home/tonghe2/train_hdf5.h5', 'w') as f:
    f['data'] = data
    f['label'] = labels  
with open('/home/tonghe2/train_hdf5.txt', 'w') as f:
    f.write('/home/tonghe2/train_hdf5.h5')
    #print('/home/tonghe2/train_hdf5.h5', file=f)

In [100]:
#for test
import Image
import numpy as np

size = 300, 300

im = Image.open('../../Images/top/4717d9eee86381b0b02831d881fd9a150c2a869f.jpg')
im = im.resize(size, Image.ANTIALIAS)
im.show()
(width, height) = im.size
#print(width, height)

im2 = np.array(im)
#print(im2.shape)
im2.astype(np.float32)

im2_ = np.rollaxis(im2,2,0)
print(im2_.shape)

#im3 = Image.fromarray(im2)
#im3.show()
#(width, height) = im3.size
#print(width, height)

im2_input = im2_[np.newaxis, :]
#print(im2_input.shape)

im4 = Image.open('../../Images/top/b5d0e05e344d0a62ceacace3725beea646f0aefa.jpg')
im4 = im4.resize(size, Image.ANTIALIAS)
im4.show()
im4 = np.array(im4)
#print(im4.shape)
im4.astype(np.float32)

im4_ = np.rollaxis(im4,2,0)
print(im4_.shape)

imx = im4_

im_final = np.concatenate((im2_, im4_, imx),axis = 0)
print(im_final.shape)

#slice for test
im1_back = im_final[6:9,:,:]
print(im1_back.shape)
im1_back = np.rollaxis(im1_back,0,3)
im1_back = Image.fromarray(im1_back)
im1_back.show()

(3, 300, 300)
(3, 300, 300)
9
(3, 300, 300)


In [10]:
size = 1, 2
print(size[1])

2
